# **yeast getting muts** 

In [3]:
#!unzip -q yeast_mut_data.zip

In [20]:
# copy the mutation dataset from home computer and paste into new text document as the file contains other irrelvant data and cant be opened
#note thersARE MULTIPLE TABLES IN THE supplemetntal fiel so I just copied the one "list SNMs.." and "fixed differecnes" as described in the file title

In [5]:
import pandas as pd
from Bio import SeqIO
import json 

** read in mutation data into df ** 

In [18]:
muts_df = pd.read_csv("Supplementary Materials 2 SNMs List.txt", sep = "\t")

In [24]:
MA_ances_muts = pd.read_csv("Supplementary Materials 1 Fixed Differences Between MA Ancestor and S288C.txt", sep = "\t")
MA_ances_muts.columns = ['Chromosome', 'Position', 'S288C', 'MA_Ancestor', 'Notes'] #remove spaces from colnames 

In [19]:
muts_df.head()

,Line ID,Chromosome,Position,Ref,Alt,Notes
0,2,3,4433,C,T,NaN
1,2,10,741951,A,G,NaN
2,2,10,741984,A,G,NaN
3,2,12,385774,A,G,NaN
4,2,12,600874,T,G,NaN


** check to make sure none of the SNMs are sites that the MA line already differ from the S2888C reference genome ** 

In [45]:
for index, row in muts_df.iterrows(): 
    if row.Position in list(MA_ances_muts.Position): # do any of the SNM pos match the fixed differences b/t MA ancestor and S2888C ref genome? 
        if row.Chromosome == int(MA_ances_muts[MA_ances_muts.Position == row.Position].Chromosome): # if yes, then is it on the same chrom? 
            print("Shite, we have overlapping muts")       

** genome data into dictionary ** 

In [50]:
!ls genome/ > yeast_genome_filenames.txt

In [60]:
#putting all the different files into a single dictionary 
yeast_genome_dict = {}
roman_transl_dict = {"I":1, "II":2, "III":3, "IV":4, "V":5, "VI":6, "VII":7, "VIII":8, "IX":9, "X":10, "XI":11, "XII":12, "XIII":13, "XIV":14, "XV":15, "XVI":16}
for filename in open("yeast_genome_filenames.txt").readlines(): 
    filename = "genome/"+filename.rstrip("\n")
    roman_chrom_n = filename.split("_")[1].replace("chr","") #extract the roman number from the file name 
    chrom = roman_transl_dict[roman_chrom_n] #cxonver to arabic numerals (clearly superior than roman) 
    for record in SeqIO.parse(open(filename), "fasta"): 
        yeast_genome_dict[str(chrom)] = record.seq

** creating the empty dictionary to fill ** 

In [76]:
bases = ["A", "T", "C", "G"]
yeast_mut_dict = {}
for b1 in bases: 
    for b2 in bases: 
        for b3 in bases: 
            triplet = b1+b2+b3
            yeast_mut_dict[triplet] = [0,{}]
            
            into_trips = []
            for b2_into in bases: 
                if b2_into != b2: 
                    into_triplet = b1+b2_into+b3
                    yeast_mut_dict[triplet][1][into_triplet] = 0 # now the dict has [XAX] = [0,{XTX:0, XGX:0, XCX:0}] format 

** using genome files to get mutation triplets ** 

In [77]:
for index, row in muts_df.iterrows(): 
    mut_pos = row.Position -1 #convert 1 to 0 base for python 
    #built in safety that the ref alligns to the reference 
    if yeast_genome_dict[str(row.Chromosome)][mut_pos]!= row.Ref: 
        print("shite")
    
    anc_triplet = str(yeast_genome_dict[str(row.Chromosome)][mut_pos-1:mut_pos+2])
    mut_triplet = anc_triplet[0]+row.Alt+anc_triplet[2]
    
    yeast_mut_dict[anc_triplet][0]+= 1
    yeast_mut_dict[anc_triplet][1][mut_triplet]+= 1


In [80]:
with open('muts_2021_01_26_yeast.txt', 'w') as outfile:
    json.dump(yeast_mut_dict, outfile)